In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sys
import os
import re
from collections import Counter
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import copy
import pickle
import re
import math

In [22]:
# Download the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/myasin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# Import the csv file with all downloaded READMES
df = pd.read_csv("../user_data_csv/csv_readme_per_user.csv")
df.head(10)

,USERNAMES,READMES
0,alexbw,Sampling Inference for Bayesian HSMMs and HMMs...
1,iassael,DEARanking \n Proposing a hybrid DEA/Polynomia...
2,bshillingford,id3tag-fix codehackathon2014 \n Team members:\...
3,hmansell,NaN
4,clementfarabet,sys \n Has moved to a more community friendly ...
5,ajabri,pytorch-maml \n This is a PyTorch implementati...
6,USERNAMES,READMES
7,coryf,homebrew \n Personal homebrew formula Sudoku C...
8,jrodrigomg,"SERVIDOR NODEJS PROYECTO 1, SISTEMAS OPERATIVO..."
9,vascofernandes,CustomersVRPF \n The project consists of two m...


In [25]:
# Apply different methods to clean the text e.g. remove appestorph and unnessary signs
def clean_df(df):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    df['READMES'] = df['READMES'].str.replace('[^a-zA-Z0-9]', ' ', regex=True).str.strip()
    df["READMES"] = df["READMES"].apply(str)
    df["READMES"] = df["READMES"].str.replace(r"[\"\',]", '')
    df["READMES"] = df["READMES"].apply(lambda x: ''.join(ch for ch in x if ch not in set(symbols)))
    df['READMES'] = df['READMES'].str.replace('\d+', '') 
    df['READMES'] = df['READMES'].str.lower()
    df = df.dropna(how='all')
    return df


In [26]:
# Remove punctiation
def remove_punctuation(df):
    df["READMES"] = df['READMES'].str.replace('[^\w\s]','')
    return df

In [27]:
# Remvoe all stopwords from the READMES column
def remove_stop_words(df):
    # import the stopwords from the nltk library
    stop_words = stopwords.words('english')
    df["READMES"] = df["READMES"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

In [28]:
# Remove all single characters from the READMES column
def remove_single_characters(df):
    df['READMES'] = df['READMES'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
    return df

In [74]:
# # Stemming the READMES form the dataframe column
# def stemming(df):
#     stemmer= PorterStemmer()
#     df['READMES'] = df['READMES'].apply(lambda x: [stemmer.stem(y) for y in x])
#     return df

In [29]:
# Call all function to clean and preprosses the READMES
def preprocess(df):
    df = clean_df(df)
    df = remove_stop_words(df)
    df = remove_punctuation(df) 
    df = remove_single_characters(df)
    df = df[df['READMES'].notna()]
    # df = stemming(df)
    df.to_csv("cleaned_df.csv", index=False)
    return df
    

In [30]:
df = preprocess(df)

/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_29818/2633761482.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df["READMES"] = df["READMES"].str.replace(r"[\"\',]", '')
/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_29818/2633761482.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['READMES'] = df['READMES'].str.replace('\d+', '')
/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_29818/1226715876.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["READMES"] = df['READMES'].str.replace('[^\w\s]','')


In [78]:
'''
def word_frequency(df):
    word_frequency_dict = {}
    for username,readme in zip(df["USERNAMES"], df["READMES"].str.lower()):
        word_frequency_dict[username] = {}
        for word in readme.split():
            if word not in word_frequency_dict[username].keys():
                word_frequency_dict[username][word] = 1
            else:
                word_frequency_dict[username][word] += 1
        word_frequency_dict[username] = (sorted(word_frequency_dict[username].items(), key=lambda x: -x[1]))
    return word_frequency_dict
'''


'\ndef word_frequency(df):\n    word_frequency_dict = {}\n    for username,readme in zip(df["USERNAMES"], df["READMES"].str.lower()):\n        word_frequency_dict[username] = {}\n        for word in readme.split():\n            if word not in word_frequency_dict[username].keys():\n                word_frequency_dict[username][word] = 1\n            else:\n                word_frequency_dict[username][word] += 1\n        word_frequency_dict[username] = (sorted(word_frequency_dict[username].items(), key=lambda x: -x[1]))\n    return word_frequency_dict\n'

In [31]:
df = pd.read_csv("cleaned_df.csv")
df = df[df['READMES'].notna()]
df.head(10)

,USERNAMES,READMES
0,alexbw,sampling inference bayesian hsmms hmms python ...
1,iassael,dearanking proposing hybrid dea polynomial int...
2,bshillingford,idtag fix codehackathon team members brendan s...
4,clementfarabet,sys moved community friendly repo xlua moved c...
5,ajabri,pytorch maml pytorch implementation supervised...
6,USERNAMES,readmes
7,coryf,homebrew personal homebrew formula sudoku curses
8,jrodrigomg,servidor nodejs proyecto sistemas operativos e...
9,vascofernandes,customersvrpf project consists two main parts ...
10,seba-1511,arduino chrono arduino based chronometer toosk...


In [ ]:
#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_text(df):
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(df['READMES'])]

# df['READMES'] = df.apply(lemmatize_text, axis=1)

In [42]:

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, stop_words='english')
vectors = vectorizer.fit_transform(df["READMES"])


In [46]:
print(vectors)

  (0, 2719)	0.007143697051585221
  (0, 8072)	0.007160569089295756
  (0, 9928)	0.018558089399799138
  (0, 18426)	0.018558089399799138
  (0, 3424)	0.01671800689227221
  (0, 3415)	0.017835238745262823
  (0, 3414)	0.01946336756489891
  (0, 6698)	0.05453568763789372
  (0, 14611)	0.05694709297664868
  (0, 16874)	0.05694709297664868
  (0, 8142)	0.03343601378454442
  (0, 1716)	0.01946336756489891
  (0, 8133)	0.01696653118326461
  (0, 8140)	0.03796472865109912
  (0, 8968)	0.06825193727687957
  (0, 7757)	0.018558089399799138
  (0, 11278)	0.014877924384745287
  (0, 13984)	0.01946336756489891
  (0, 13620)	0.017521808671857198
  (0, 529)	0.01898236432554956
  (0, 16574)	0.01898236432554956
  (0, 16419)	0.018178562545964574
  (0, 18403)	0.011117961124622424
  (0, 16161)	0.010223975048599648
  (0, 17596)	0.01898236432554956
  :	:
  (570, 18648)	0.0064334198875075676
  (570, 5386)	0.005924686937199583
  (570, 20379)	0.013263669373786912
  (570, 1037)	0.02246387785374535
  (570, 3057)	0.018117235916839

In [33]:
feature_names = vectorizer.get_feature_names()

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
termfreq = vectorizer.vocabulary_

In [35]:
termfreq

{'sampling': 17766,
 'inference': 9537,
 'bayesian': 1515,
 'python': 16022,
 'library': 11202,
 'approximate': 873,
 'unsupervised': 21294,
 'hidden': 8690,
 'markov': 12151,
 'models': 12727,
 'explicit': 6671,
 'duration': 5837,
 'semi': 18103,
 'focusing': 7415,
 'nonparametric': 13557,
 'extensions': 6734,
 'hmm': 8747,
 'weak': 22570,
 'limit': 11397,
 'approximation': 877,
 'plugins': 14990,
 'extend': 6724,
 'functionality': 7806,
 'factorial': 6794,
 'autoregressive': 1201,
 'collapsed': 3018,
 'run': 17454,
 'parallel': 14479,
 'multiple': 12961,
 'cores': 3892,
 'machines': 11915,
 'ec': 5901,
 'using': 21811,
 'ipython': 10250,
 'excellent': 6544,
 'module': 12807,
 'someday': 18995,
 'document': 5623,
 'installing': 9989,
 'clone': 2705,
 'dependencies': 4950,
 'git': 8058,
 'recursive': 16612,
 'github': 8123,
 'com': 3057,
 'depends': 4993,
 'numpy': 13792,
 'scipy': 17884,
 'visualization': 22362,
 'matplotlib': 12216,
 'disabling': 5390,
 'assertions': 1037,
 'speed': 

In [36]:
query = "Computer vison"

In [37]:
query_vec = vectorizer.transform([query])

In [38]:
results = cosine_similarity(vectors,query_vec).reshape((-1,))

In [39]:
results.shape

(571,)

In [41]:
# Print Top 10 results
for i in results.argsort()[-30:][::-1]:
    print(df.iloc[i,0], results[i])


themadarchitect 0.08030971134256305
edgarriba 0.0700675398708982
carwyn 0.05981791624939909
amirsaffari 0.05413815485446251
intermilan 0.053504328397584074
rAm1n 0.05188081226160316
mirceamironenco 0.04800960624249392
spillai 0.04773963550622932
SalemAmeen 0.04605712317886973
vchoutas 0.04528432939435094
warmlogic 0.04354476963040358
alxndrkalinin 0.04129183729083068
kaustubhn 0.03999019344990282
gr33ndata 0.0386747724032597
durandtibo 0.037604097083563666
valthom 0.03615526379390335
NikhilDoWhile 0.03472763994686516
bazilas 0.033654872189445156
AdrianJohnston 0.03297147125013977
meetps 0.03148692239904069
meetps 0.03148692239904069
iassael 0.031155285245190886
iassael 0.031155285245190886
mlagunas 0.030260912937024664
jordanbangia 0.02824283598743966
cvondrick 0.028147854288722175
adikhosla 0.027608720894017138
aenrichus 0.02739439669945395
mrecachinas 0.027015436908416235
hfslyc 0.02678419732815575


In [91]:
# Print Top 10 results
for i in results.argsort()[-10:][::-1]:
    print(df.iloc[i,0], results[i])

themadarchitect 0.07904647986752661
edgarriba 0.06879900769740685
intermilan 0.05401933393845209
rAm1n 0.05207870749500482
amirsaffari 0.05203149885115723
mirceamironenco 0.0487829589616814
SalemAmeen 0.04647568237169033
alxndrkalinin 0.04030317722675246
gr33ndata 0.03867786058278458
valthom 0.03691244395486756


In [106]:

recommended_users_plus_score = {}
query_vec = vectorizer.transform([query])
results = cosine_similarity(vectors,query_vec).reshape((-1,))
for name in results.argsort()[-10:][::-1]:
    recommended_users_plus_score[df.iloc[name,0]] = results[name]
print(recommended_users_plus_score)

{'themadarchitect': 0.07904647986752661, 'edgarriba': 0.06879900769740685, 'intermilan': 0.05401933393845209, 'rAm1n': 0.05207870749500482, 'amirsaffari': 0.05203149885115723, 'mirceamironenco': 0.0487829589616814, 'SalemAmeen': 0.04647568237169033, 'alxndrkalinin': 0.04030317722675246, 'gr33ndata': 0.03867786058278458, 'valthom': 0.03691244395486756}


In [109]:
def vectoriz_dataframe(df):

    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0.01, stop_words='english')
    vectorize_dataframe = vectorizer.fit_transform(df["READMES"])

    return vectorize_dataframe

In [111]:
def jobtitle_recommended(vectorize_dataframe, query):
    
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(vectorize_dataframe, query_vec).reshape((-1,))
    
    for i in results.argsort()[-10:][::-1]:
        print(df.iloc[i,0], results[i])
        
        recommended_users_plus_score = results[i]
    print(recommended_users_plus_score)   
    return recommended_users_plus_score

In [ ]:
score_per_user = jobtitle_recommended(vectorize_dataframe, query)